In [1]:
# !pip install transformers accelerate
from transformers import EarlyStoppingCallback
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import Dataset
# %pip install evaluate
import evaluate
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys

# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 
from commitfit import get_templated_dataset,sample_dataset


root
autodl-tmp
CommitFit


In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
# df.fillna('', inplace=True)
label2id={'C':0, 'A':1, 'P':2}
df = df.rename(columns={'labels':'label','msgs':'text','commit_diff':'diffs'})
df = df.replace({"label": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

,repo,commit,label,text,diffs,feature
user,,,,,,
ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,Perfective,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,Adaptive,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,Adaptive,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ..."
ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,Corrective,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,..."
ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,Corrective,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,Perfective,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,Perfective,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,..."
hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,Perfective,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17..."


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=10, random_state=42))

/tmp/ipykernel_1859/1938515031.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=10, random_state=42))


In [5]:
# train.to_csv('/root/autodl-tmp/CommitFit/dataset/Ghadhab/10_train.csv')
# test.to_csv('/root/autodl-tmp/CommitFit/dataset/Ghadhab/test.csv')
# val.to_csv('/root/autodl-tmp/CommitFit/dataset/Ghadhab/val.csv')

# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/Ghadhab/10_train.csv')
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/Ghadhab/test.csv')
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/Ghadhab/val.csv')

In [6]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [7]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [8]:
train_code_change = list(train['diffs'].astype(str))
test_code_change = list(test['diffs'].astype(str))

In [9]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['repo', 'commit', 'label', 'text', 'diffs', 'feature', 'user'],
    num_rows: 54
})

In [10]:
len(train)

30

In [11]:
train['label'].value_counts()

label
Adaptive      10
Corrective    10
Perfective    10
Name: count, dtype: int64

In [12]:
test['label'].value_counts()

label
Perfective    100
Corrective     89
Adaptive       79
Name: count, dtype: int64

In [13]:
len(train_dataset)

54

In [14]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='macro')
    recall_score = metrics.recall_score(y_test,y_pred,average='macro')
    f1_score = metrics.f1_score(y_test,y_pred,average='macro')
    
    # 混淆矩阵
    cm = metrics.confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", cm)
    print(metrics.classification_report(y_test, y_pred, digits=4))
    # return {"classification_report": classification_report}
    return {"classification_report":metrics.classification_report(y_test, y_pred, digits=4),"Confusion Matrix":cm,"precision": round(precision_score, 4),"recall": round(recall_score, 4), "f1":round(f1_score, 4), "accuracy": round(accuracy_score, 4)}

In [15]:
model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"

In [16]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [21]:
trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=20,
    batch_size = 32,
    num_epochs=10
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2025-12-11 16:15:07,340] A new study created in memory with name: no-name-1bcd0df4-bb0b-4fdc-ab67-a4be31cc9359
Trial: {'learning_rate': 0.00030252017637320437}
model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2160
  Num epochs = 10
  Total optimization steps = 680
  Total train batch size = 32


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

***** Running evaluation *****
[I 2025-12-11 16:35:36,598] Trial 9 finished with value: 0.653 and parameters: {'learning_rate': 7.144200448545207e-05}. Best is trial 9 with value: 0.653.


Confusion Matrix:
 [[59 10 10]
 [17 72  0]
 [41 15 44]]
              precision    recall  f1-score   support

    Adaptive     0.5043    0.7468    0.6020        79
  Corrective     0.7423    0.8090    0.7742        89
  Perfective     0.8148    0.4400    0.5714       100

    accuracy                         0.6530       268
   macro avg     0.6871    0.6653    0.6492       268
weighted avg     0.6992    0.6530    0.6478       268

+++++++++++ {'classification_report': '              precision    recall  f1-score   support\n\n    Adaptive     0.5043    0.7468    0.6020        79\n  Corrective     0.7423    0.8090    0.7742        89\n  Perfective     0.8148    0.4400    0.5714       100\n\n    accuracy                         0.6530       268\n   macro avg     0.6871    0.6653    0.6492       268\nweighted avg     0.6992    0.6530    0.6478       268\n', 'Confusion Matrix': array([[59, 10, 10],
       [17, 72,  0],
       [41, 15, 44]]), 'precision': 0.6871, 'recall': 0.6653, 'f1': 0.

In [22]:
# best_run

In [23]:
best_run.hyperparameters

{'learning_rate': 7.144200448545207e-05}

In [24]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2160
  Num epochs = 10
  Total optimization steps = 680
  Total train batch size = 32


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68 [00:00<?, ?it/s]

In [25]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

***** Running evaluation *****


Confusion Matrix:
 [[60  8 11]
 [21 67  1]
 [37  7 56]]
              precision    recall  f1-score   support

    Adaptive     0.5085    0.7595    0.6091        79
  Corrective     0.8171    0.7528    0.7836        89
  Perfective     0.8235    0.5600    0.6667       100

    accuracy                         0.6828       268
   macro avg     0.7164    0.6908    0.6865       268
weighted avg     0.7285    0.6828    0.6885       268



{'classification_report': '              precision    recall  f1-score   support\n\n    Adaptive     0.5085    0.7595    0.6091        79\n  Corrective     0.8171    0.7528    0.7836        89\n  Perfective     0.8235    0.5600    0.6667       100\n\n    accuracy                         0.6828       268\n   macro avg     0.7164    0.6908    0.6865       268\nweighted avg     0.7285    0.6828    0.6885       268\n',
 'Confusion Matrix': array([[60,  8, 11],
        [21, 67,  1],
        [37,  7, 56]]),
 'precision': 0.7164,
 'recall': 0.6908,
 'f1': 0.6865,
 'accuracy': 0.6828}

In [26]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{fewshot_metrics}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
